In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data  #导入数据
#下面这一行跟分类器中的是一样的
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)


def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={xs:v_xs,keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.arfmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#定义卷积神经网络层了
def conv2d(x,W):   # x代表输入的值，或者是图片的值。 W就是weight
    #strides=[1,x轴移动距离，y轴移动距离，1]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')#padding就是选择是否使用填充的扫描方式
    #这里就是返回tensorflow中2维的卷积神经网络。 
    
    #第三个参数就是设置步长，在tensorflow中它是一个长度为4的列表。在这个列表中第一个以及最后一个元素都是必须等于1的。列表中间的两个
    #参数分别是代表在x轴，y轴上移动的距离
    
def max_pool_2*2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #使用最大池化的方法， ksize=[1,height,width,1]这个是池化窗口的大小

#设置placeholder来进行数据的传输
xs = tf.placeholder(tf.float32,[None,784])
ys = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)


#设置代价误差函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
# tf.summary.scalar('loss',cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#全局变量的初始化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    batch_xs,batch_ys = mnist.train.next_batch(100)
    sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys,keep_prob:0.5})
    if i % 50 == 0:
        print(compute_accuracy(mnist.test.images[:1000],mnist.test.labels[:1000]))
